In [1]:
import pandas as pd

UNIT_CONVERTER = 24.5
data = pd.read_excel('media/data/order2024.xlsx', engine="openpyxl")

    # Rename columns
data = data.rename(columns={
        "กำหนดส่ง": "due_date",
        "แผ่นหน้า": "front_sheet",
        "ลอน C": "c_wave",
        "แผ่นกลาง": "middle_sheet",
        "ลอน B": "b_wave",
        "แผ่นหลัง": "back_sheet",
        "จน.ชั้น": "level",
        "กว้างผลิต": "width",
        "ยาวผลิต": "length",
        "ทับเส้นซ้าย": "left_edge_cut",
        "ทับเส้นกลาง": "middle_edge_cut",
        "ทับเส้นขวา": "right_edge_cut",
        "เลขที่ใบสั่งขาย": "order_number",
        "ชนิดส่วนประกอบ": "component_type",
        "จำนวนสั่งขาย": "quantity",
        "จำนวนสั่งผลิต": "production_quantity",localhost
        "ประเภททับเส้น": "edge_type",
        "สถานะใบสั่ง": "order_status",
        "% ที่เกิน": "excess_percentage"
    })



In [2]:
import uuid

for index, row in data.iterrows():
    data.at[index, 'width'] = round(row["width"] / UNIT_CONVERTER, 4)
    data.at[index, 'length'] = round(row["length"] / UNIT_CONVERTER, 4)
    data.at[index, 'id'] = uuid.uuid4()


/tmp/ipykernel_9298/1617615471.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11.102' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'width'] = round(row["width"] / UNIT_CONVERTER, 4)
/tmp/ipykernel_9298/1617615471.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.0204' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'length'] = round(row["length"] / UNIT_CONVERTER, 4)


In [3]:
start_date = pd.to_datetime('2024-08-1')
stop_date = pd.to_datetime('2024-08-7')


dated_data = data[(data['due_date'] >= start_date) & (data['due_date'] <= stop_date)].reset_index(drop=True)


In [4]:
import random

LEGACY_FILTER = [
    "front_sheet",
    "c_wave",
    "middle_sheet",
    "b_wave",
    "back_sheet",
]

def legacy_filter_order(data):
        legacy_filters = LEGACY_FILTER
        ordplan = pd.DataFrame(None)
        best_index=0
        most_compat_plan = 0
        indices = list(range(len(data)))
        random.shuffle(indices)

        for index in indices:
            init_order = data.iloc[index]
            # Create a mask for matching orders using all legacy filters
            mask = (data[legacy_filters].eq(init_order[legacy_filters])).all(axis=1)
            # Apply the mask and reset the index
            ordplan = data.loc[mask].reset_index(drop=True)
            if len(ordplan)>most_compat_plan:
                best_index=index
                most_compat_plan=len(ordplan)

        init_order = data.iloc[best_index]
        mask = (data[legacy_filters].eq(init_order[legacy_filters])).all(axis=1)
        ordplan = data.loc[mask].reset_index(drop=True)
        data = ordplan
        return data

In [5]:
filtered_data = legacy_filter_order(dated_data)


In [6]:
from icecream import ic
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict
from numpy import roll
import pandas as pd



In [7]:
# from modules.ordplan import ORD

# legacy_data = ORD(filtered_data, deadline_range = 30, size=temp_size).get()



In [8]:

from ortools.linear_solver import pywraplp
# from ordplan_project.settings import ROLL_PAPER

ROLL_PAPER = [66, 68, 70, 73, 74, 75, 79, 82, 85, 88, 91, 93, 95, 97]
@dataclass
class LP:
    results: Dict[str, Any]
    def run(self):
        roll_paper = ROLL_PAPER
        results = self.results
        if results is None:
            raise ValueError('Results is empty!')
        solver = pywraplp.Solver.CreateSolver("SAT")

        variables = {
            f'{roll}' :  solver.IntVar(0, 1,f'{roll}')
            for roll in roll_paper
        }

        solver.Add(
            sum(variables[f'{roll}'] for roll in roll_paper) <= 1
        )

        solver.Add(
            sum(roll * variables[f'{roll}'] for roll in roll_paper) - results['fitness'] <= 3
        )

        solver.Add(
            sum(roll * variables[f'{roll}'] for roll in roll_paper) - results['fitness'] >= 1
        )

        solver.Minimize(
             sum(roll * variables[f'{roll}'] for roll in roll_paper) - results['fitness']
        )

        status = solver.Solve()

        if status != pywraplp.Solver.OPTIMAL:
            self.output = None
            return self

        output = {'new_trim': float, 'old_fitness': float, 'new_roll':int}
        new_trim = solver.Objective().Value()

        output['new_trim']= new_trim
        output['old_fitness']= results['fitness']
        output['new_roll'] = sum(variables[f'{roll}'].solution_value()*roll for roll in roll_paper)
        self.output = output
        return self

    def get(self):
        return self.output


def main():
    results = {'fitness': 69.1836}
    model = LP(results)
    model.run()
    print(model.get())

if __name__ == "__main__":
    main()


None


In [9]:
from django.conf import settings

from dataclasses import dataclass, field
from pandas import DataFrame
import pygad
import numpy
import pandas as pd
from typing import Callable, Dict, Any, List, Optional
from icecream import ic

from order_optimization.container import ModelInterface

from ordplan_project.settings import MIN_TRIM,PENALTY_VALUE


@dataclass
class GA(ModelInterface):
    orders: DataFrame
    size: float = 66
    num_generations: int = 50
    out_range: int = 6
    showOutput: bool = False
    save_solutions: bool = False
    showZero: bool = False
    selector: Dict[str, Any] | None = None
    set_progress: Callable | None = None
    current_generation: int = 0
    _penalty:int = 0 
    _penalty_value:int = PENALTY_VALUE
    blade:Optional[int] = None
    seed:Optional[int] = None
    parent_selection_type:str ="tournament"
    crossover_type:str ="uniform"
    mutation_probability:Optional[List[int]]=None
    mutation_percent_genes:List[int] = field(default_factory=lambda: [25,5])
    crossover_probability:float=None
    
    def __post_init__(self):
        if self.orders is None:
            raise ValueError("Orders is empty!")
        self.orders = self.orders[self.orders['quantity'] > 0].reset_index(drop=True)
        self._paper_size  = self.size
        self.model = pygad.GA(
            num_generations=self.num_generations,
            num_parents_mating=60,
            fitness_func=self.fitness_function,
            sol_per_pop=120,
            num_genes=len(self.orders),

            gene_type=int,
            init_range_low=0,
            init_range_high=self.out_range,
            parent_selection_type=self.parent_selection_type,
            crossover_type=self.crossover_type,
            mutation_type="adaptive",
            mutation_probability=self.mutation_probability,
            mutation_percent_genes=self.mutation_percent_genes,
            crossover_probability=self.crossover_probability,
            on_generation=self.on_gen,
            save_solutions=self.save_solutions,
            stop_criteria="saturate_7",
            suppress_warnings=True,
            random_seed=self.seed
        )

        
    def paper_type_logic(self, solution):
        init_type = None
        orders = self.orders
        match orders["edge_type"][self.get_first_solution(solution)]:
            case "X":
                init_type = 1
            case "N":
                init_type = 2
            case "W":
                init_type = 2

        if init_type is not None:
            for index, out in enumerate(solution):
                if out >= 1:
                    match init_type:
                        case 1:
                            if orders["edge_type"][index] not in [
                                "X",
                                "Y",
                            ]:  # Changed OR to AND condition
                                self._penalty += self._penalty_value
                        case 2:
                            if orders["edge_type"][index] == "X":
                                self._penalty += self._penalty_value

    def least_order_logic(self, solution):
        init_order = None
        orders = self.orders

        init_order = orders["quantity"][self.get_first_solution(solution)]

        for index, out in enumerate(solution):
            if out >= 1 and orders["quantity"][index] < init_order:
                self._penalty += self._penalty_value

    @staticmethod
    def get_first_solution(solution) -> int:
        for index, out in enumerate(solution):
            if out >= 1:
                return index
        return 0

    def paper_out_logic(self, solution):
        if sum(solution) > 5:
            if sum(solution) <= 6:
                init = 0
                for index, out in enumerate(solution):
                    if out>=1:
                        if self.orders['edge_type'][index]=='X' and init==0:
                            init = 1
                            continue
                        if self.orders['edge_type'][index]=='Y' and init==1:
                            return                
            
            self._penalty += self._penalty_value * sum(solution)  # ยิ่งเกิน ยิ่ง _penaltyเยอะ
        
        
        order_length = 0
        for index, out in enumerate(solution):
            if out >= 1:
                order_length += 1
        if order_length > 2:
            self._penalty += self._penalty_value * order_length  # ยิ่งเกิน ยิ่ง _penaltyเยอะ

    def paper_size_logic(self, _output):
        if _output > self._paper_size :  # ถ้าผลรวมมีค่ามากกว่า roll กำหนดขึ้น _penalty
            self._penalty += self._penalty_value * (
                _output - self._paper_size 
            )  # ยิ่งเกิน ยิ่ง _penaltyเยอะ

    def paper_trim_logic(self, _fitness_values):
        if abs(_fitness_values) <= MIN_TRIM:  # ถ้าผลรวมมีค่าน้อยกว่า _penalty > เงื่อนไขบริษัท
            self._penalty += self._penalty_value

    def selector_logic(self, solution: List[int])->List[int]:
        if self.selector is None:
            return solution
        
        try:
            solution[0] = self.selector["out"] #lock the first to be out (the first order is also the selector, manage by ORD)
        except KeyError:
            pass

        if solution[0] == 0: 
            solution[0] += 1

        return solution

    def fitness_function(self, ga_instance, solution, solution_idx):
        self._penalty = 0

        solution = self.selector_logic(solution)

        self.paper_type_logic(solution)

        self.least_order_logic(solution)

        self.paper_out_logic(solution)

        _output = numpy.sum(solution * self.orders["width"])  # ผลรวมของตัดกว้างทั้งหมด

        _fitness_values = -self._paper_size  + _output  # ผลต่างของกระดาษที่มีกับออเดอร์ ยิ่งเยอะยิ่งดี
        self.paper_trim_logic(_fitness_values)
        return _fitness_values - self._penalty  # ลบด้วย _penalty

    def on_gen(self, ga_instance):

        self.current_generation += 1
        if self.set_progress:
            progress = (self.current_generation / self.num_generations) * 100
            self.set_progress(progress)

        orders = self.orders

        solution = ga_instance.best_solution()[0]

        _output = pd.DataFrame(
            {   
                "id": orders['id'].unique(),
                "blade": 0,
                "order_number": orders["order_number"],
                "num_orders": orders["quantity"],
                "component_type": orders["component_type"],
                "cut_width": orders["width"],
                "cut_len": orders["length"],
                "type": orders["edge_type"],
                "deadline": orders["due_date"],
                "front_sheet": orders["front_sheet"],
                "c_wave": orders["c_wave"],
                "middle_sheet": orders["middle_sheet"],
                "b_wave": orders["b_wave"],
                "back_sheet": orders["back_sheet"],
                "num_layers": orders["level"],
                "left_line": orders["left_edge_cut"],
                "center_line": orders["middle_edge_cut"],
                "right_line": orders["right_edge_cut"],
                "out": solution,
            }
        )

        if not self.showZero:
            _output = _output[_output["out"] >= 1]
        _output = _output.reset_index(drop=True)


        _output = self.blade_logic(_output)

        self._fitness_values = ga_instance.best_solution()[1]
        self._output = _output

        if self.showOutput:
            self.show(ga_instance, _output)

    def blade_logic(self, output: DataFrame) -> DataFrame:
        blade_list: List[Dict[str,int]] = []
        for idx in output.index:
            blade_val = idx+1
            if self.blade is not None:
                blade_val = self.blade
            blade_list.append({"blade": blade_val})

        blade_df = pd.DataFrame(blade_list)
        output = pd.concat([output, blade_df], axis=1)
        return output

    def show(self, ga_instance, _output):
        _paper_size  = self._paper_size 
        print("Generation : ", ga_instance.generations_completed)
        print("Solution :")

        with pd.option_context(
            "display.max_columns",
            None,
            "display.width",
            None,
            "display.colheader_justify",
            "left",
        ):
            print(_output.to_string(index=False))

        print("Roll :", _paper_size )
        print("Used :", _paper_size  + self._fitness_values)
        print("Trim :", abs(self._fitness_values))
        print("\n")

    @property
    def output(self) -> DataFrame:
        return self._output
    
    @property
    def fitness_values(self) -> float:
        return self._fitness_values

    @property
    def penalty(self) -> int:
        return self._penalty
    
    @penalty.setter
    def penalty(self, penalty:int) -> None:
        self._penalty = penalty

    @property
    def PAPER_SIZE(self) -> float:
        return self._paper_size 

    @PAPER_SIZE.setter
    def PAPER_SIZE (self, size: float):
        self._paper_size  = size
    
    @property
    def run(self) -> Callable:
        return self.model.run


In [10]:
from typing import List, Any ,Optional

@dataclass
class HD:
    ROLL_PAPER = [66, 68, 70, 73, 74, 75, 79, 82, 85, 88, 91, 93, 95, 97]

    x:int = 10 
    h_type:str = 'ff'

    def __post_init__(self):
        self.temp_size = min(self.ROLL_PAPER)
        self.ff_list = []
        self.ffa_list = []
        self.ffd_list = []

        match(self.h_type):
            case 'ff':
                self.ff_list = [[] for _ in range(self.x)]
            case 'ffa':
                self.ffa_list = [[] for _ in range(self.x)]

        ff_list = self.ff_list
        ffa_list = self.ffa_list
        ffd_list = self.ffd_list
        for item, id in zip(filtered_data['width'], filtered_data['id']):
            ff_list = self.first_fit(ff_list, item, id)
        
        # print(width_sum_formatter(ff_list))
        ff_df = self.df_formatter(ff_list)
        
        desc_filtered_data = filtered_data.sort_values('width', ascending=False)
        for item, id in zip(desc_filtered_data['width'], desc_filtered_data['id']):
            ffd_list = self.first_fit(ffd_list, item, id)
        
        # print(width_sum_formatter(ffd_list))
        ffd_df = self.df_formatter(ffd_list)
        
        asc_filtered_data = filtered_data.sort_values('width', ascending=True)
        for item, id in zip(asc_filtered_data['width'], asc_filtered_data['id']):
            ffa_list = self.first_fit(ffa_list, item, id)
            
        # print(width_sum_formatter(ffa_list))
        ffa_df = self.df_formatter(ffa_list)
        
        heuristic_data_id = pd.concat([ffd_df, ff_df, ffa_df])
        heuristic_data_id = heuristic_data_id.drop_duplicates('id').reset_index(drop=True)
        
        self.heuristic_data = filtered_data[filtered_data['id'].isin(heuristic_data_id['id'])].reset_index(drop=True)
    
    
    def is_fit(self, data_list, item):
        return sum([row[0] for row in data_list]) + item - self.temp_size < 0

    @staticmethod
    def get_width(data_list):
        return [row[0] for row in data_list]
    
    
    def width_sum_formatter(self, data_list):
        output = ""
        for index, _ in enumerate(data_list):
            output+= str(round(sum(self.get_width(data_list[index])),4))+" | "
        return output

    @staticmethod
    def df_formatter(data_list):
        combined_data = []
        for data in data_list:
            combined_data += data
        return pd.DataFrame(combined_data, columns=['width', 'id'])


    def first_fit(self,data_list, item, id):
        for data in data_list:
            if self.is_fit(data, item):
                data.append((item, id))
                return data_list
        return data_list

    def get(self) -> pd.DataFrame:    
        return self.heuristic_data

In [11]:
from ordplan_project.settings import MAX_TRIM, MIN_TRIM
from typing import Dict, Any

def is_trim_fit(trim: float):
    """
    Check if trim exceed min/max tirm.
    """
    return trim <= MAX_TRIM and trim >= MIN_TRIM


def handle_switcher(used) -> Dict[str,Any]:
    if is_trim_fit(used):
        return
    switcher = LP({'fitness': used}).run().get()
    # ic(switcher,used)
    return switcher

def update_acc_list(acc_list,ga_instance,hd_instance,elapsed_times):
        temp_acc_str = ""
        temp_acc_str+='Parameters -> '
        
        parameters = (
            "parent:" +ga_instance.parent_selection_type + " | " +
            "co proba:" +str(ga_instance.crossover_probability) + " | " +
            "crossover:" +ga_instance.crossover_type + " | " +
            str(ga_instance.mutation_probability) + " | " +
            str(ga_instance.mutation_percent_genes) + " | " +
            "heu type:" +hd_instance.h_type +"\n"
        )
        
        temp_acc_str+=parameters
        
        accuracy = (passed_trim / count) * 100.00 if count != 0 else 0.0
        temp_acc_str += "Accuracy -> {:.2f}%".format(accuracy)
    
        total_seconds = sum(elapsed_times)
        
        minutes, seconds = divmod(total_seconds, 60)
        
        time_spent_str = f"{int(minutes)}:{int(seconds):02d}"
        
        temp_acc_str += f"\nTime Spent -> {time_spent_str}"

        temp_acc_str += "\n"
        
        acc_list.append(temp_acc_str)

In [14]:
from tqdm import tqdm
import itertools
import time

import enlighten


crossover_types = ["two_points", "uniform"]
mutation_proba = [None,[0.25, 0.05]]
crossover_proba = [None, 0.65]
acc_list = []

x = 20
count = 100

hd_instances = []
h_types = ["ff", "ffa"]
for h_type in h_types:
    hd_instance = HD(h_type=h_type, x=x)
    hd_instances.append(hd_instance)

temp_size = hd_instance.temp_size

manager = enlighten.get_manager()
types = manager.counter(total=len(list((itertools.product(crossover_types, mutation_proba, crossover_proba, hd_instances)))), desc='Types', unit='it')
counts = manager.counter(total=count*len(list((itertools.product(crossover_types, mutation_proba, crossover_proba, hd_instances)))), desc='Count', unit='it')

for crossover_prob, crossover, mutation, hd_instance in itertools.product(crossover_proba, crossover_types, mutation_proba,hd_instances):
    heuristic_data = hd_instance.get()
    elapsed_times = []
    passed_trim = 0
    types.update()
    for i in range(count):
        counts.update()
        start_time = time.time()
        ga_instance = GA(
            heuristic_data,
            size=temp_size,
            num_generations=50,
            out_range=5,
            seed=i,
            parent_selection_type="tournament",
            crossover_type=crossover,
            mutation_probability=mutation,
            crossover_probability=crossover_prob
        )
        ga_instance.run()
        size = ga_instance.PAPER_SIZE
        trim = abs(ga_instance.fitness_values)
        used = ga_instance.PAPER_SIZE+ga_instance.fitness_values
        switcher = handle_switcher(used)
    
        if switcher is not None:
            size = switcher["new_roll"]
            trim = switcher["new_trim"]
    
        # print(size)
        # print(used)
        # print(trim)
        if trim < 3:
            passed_trim+=1
        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_times.append(elapsed_time)


    update_acc_list(acc_list,ga_instance,hd_instance,elapsed_times)


    

/tmp/ipykernel_9298/1734814982.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  heuristic_data_id = pd.concat([ffd_df, ff_df, ffa_df])
/tmp/ipykernel_9298/1734814982.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  heuristic_data_id = pd.concat([ffd_df, ff_df, ffa_df])


In [15]:
for i in acc_list:
    print(i)

Parameters -> parent:tournament | co proba:None | crossover:two_points | None | [25, 5] | heu type:ff
Accuracy -> 67.00%
Time Spent -> 9:56

Parameters -> parent:tournament | co proba:None | crossover:two_points | None | [25, 5] | heu type:ffa
Accuracy -> 55.00%
Time Spent -> 10:29

Parameters -> parent:tournament | co proba:None | crossover:two_points | [0.25, 0.05] | [25, 5] | heu type:ff
Accuracy -> 56.00%
Time Spent -> 9:14

Parameters -> parent:tournament | co proba:None | crossover:two_points | [0.25, 0.05] | [25, 5] | heu type:ffa
Accuracy -> 70.00%
Time Spent -> 10:43

Parameters -> parent:tournament | co proba:None | crossover:uniform | None | [25, 5] | heu type:ff
Accuracy -> 55.00%
Time Spent -> 8:15

Parameters -> parent:tournament | co proba:None | crossover:uniform | None | [25, 5] | heu type:ffa
Accuracy -> 58.00%
Time Spent -> 9:12

Parameters -> parent:tournament | co proba:None | crossover:uniform | [0.25, 0.05] | [25, 5] | heu type:ff
Accuracy -> 47.00%
Time Spent ->

In [16]:
!jupyter nbconvert --to script ord-dev.ipynb

[NbConvertApp] Converting notebook ord-dev.ipynb to script
[NbConvertApp] Writing 20210 bytes to ord-dev.py
